less is more - kaggle santander

# start

return to four models

In [1]:
PRFX='0319_5'
trntmstmp=1584412344
valtmstmp=1584626395
import datetime
[datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)]

['2020-03-17 02:32:24', '2020-03-19 13:59:55']

In [2]:
grand_total=1.5e8

# setup

In [3]:
from pathlib import Path
import dask
print('dask.__version__', dask.__version__)
import xgboost as xgb
import dask_xgboost
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from dask.distributed import Client
import pickle
import lightgbm as lgb
from tqdm import tqdm

import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwing_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_follower_cnt',
'u2_following_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]

cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)

dask.__version__ 2.12.0


# prepare data

## get maps from dfval

In [4]:
# dfval=dd.read_csv(
#     f'{p_in}/val_{valtmstmp}.tsv',
#     sep='\x01', header=None, names=cols_val,)
# lang2cnt=dict(dfval.lang.value_counts().compute())
# ttl=sum(lang2cnt.values())
# lang2perc={k:v/ttl for k,v in lang2cnt.items()}
langs=['D3164C7FBCF2565DDF915B1B3AEFB1DC', '22C448FF81263D4BAF2A176145EE9EAD', '06D61DCBBE938971E1EA0C38BD9B5446', 'ECED8A16BE2A5E8871FD55F4842F16B1', 'B9175601E87101A984A50F8A62A1C374', '4DC22C3F31C5C43721E6B5815A595ED6', '167115458A0DBDFF7E9C0C53A83BAC9B', '125C57F4FA6D4E110983FB11B52EFD4E', '022EC308651FACB02794A8147AEE1B78', 'FA3F382BC409C271E3D6EAF8BE4648DD', '9BF3403E0EB7EA8A256DA9019C0B0716', '975B38F44D65EE42A547283787FF5A21', '2996EB2FE8162C076D070A4C8D6532CD', 'FF60A88F53E63000266F8B9149E35AD9', '717293301FE296B0B61950D041485825', '3E16B11B7ADE3A22DDFC4423FBCEAD5D', '3820C29CBCA409A33BADF68852057C4A', '9ECD42BC079C20F156F53CB3B99E600E', '76B8A9C3013AE6414A3E6012413CDC3B', 'AEF22666801F0A5846D853B9CEB2E327', '190BA7DA361BC06BC1D7E824C378064D', '1FFD2FE4297F5E70EBC6C3230D95CB9C', 'A0C7021AD8299ADF0C9EBE326C115F6F', 'D413F5FE5236E5650A46FD983AB39212', '48236EC80FDDDFADE99420ABC9210DDF', '691890251F2B9FF922BE6D3699ABEFD2', '920502FAA080485768AA89BC96A55C47', '0331BF70E606D62D92C96CE9AD71A7CF', '89616CFF8EC8637092F885C7EFF43D74', '06BEAB41D66CCFF329D1ED8BA120A6C2', '60FBA0E834CC59D647C3599AD763FFDF', 'C7A400D9AD489ACF673CF12FBB80AAE5', 'E59EF8BB86A6D815331DDF4C467CE0C7', '4249CE88433AEA3F8DCEECF008B3CB95', '544FA32458C903F1125FE6598300A047', 'CB11E9CF42BD0A1BAD5E27BF3422D99D', '3A85BCEC571C3F5AB1069E4924189177', 'FF7EABB5A382356D54D9C41BA0125E09', 'E7BB61D2A87C1E72DF1C7BC292B86A1C', '69C4A33B9AD29AF883D60BA61CC08702', 'F4FD40A716F1572C9A28E9CAA58BE3A5', '259A6F6DFD672CB1F883CBEC01B99F2D', '6431A618DCF7F4CB7F62A95A39BAB77A', '3121F7240D488F74EEED9312E174B217', '54208B51D44E7D91DC2F3DD02ADEDEC2', 'DBEEFB80F8A314311E2B4BD593E11DFE', 'B6D90127A09AB1229731898AEF9D4D7C', 'F3E1016563360F9434FA986CA86C249C', '2216D01F7B48554E4211021A46816FCF', '1BC639981AE88E09129594B11F894A21', 'C942E369C88CE7C56E69A84D04319FF0', '57ADD4576E2AD6648E9B2DE32F3462A5', 'AC1F0671A4B0D5B8112F87DE7B490E6D', 'C2EF5FABE7619D8A333D5F0FF76E1BFA', 'AA0254541959271ED3453119B787D0C3', 'A6B70CDF8C7B934D4A218CA9B6B7FDB4', 'BF477808A37E3E4E9C5D9F1839E8519E', '97F81BD92A1ACA3F1F43C154E689350F', '60A3DB168094D41241E45E0DE3539BC0', '12D8CEB94F89D11D7EB95EAE9689B009', 'F73266A79468BB89C4325FDEDB0B533C', '4690215948DBF6872B8ED1C2BC87B17E', 'D18801336202297E6484F634CAC6592E', '2E18F6F53E3CF073911AF0A93BBE5373', 'B2235C8B73239FDC5780DD132419833A', '515E873C86EE1577E75FA2387B7FA59E',]

# dfval.twttyp.value_counts().compute()
# TopLevel    2932
# Retweet      994
# Quote        213
twttyps=['TopLevel','Retweet','Quote']

In [5]:
def prp_df(df, istrn=True):
    df['len_toks']=df.toks.apply(len)
    for media in ['Photo', 'Video', 'GIF']:
        df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x)
    for col in ['hshtgs', 'links', 'domns',]:
        df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0)
    tm_dt=pd.to_datetime(df.tm, unit='s')
    df['tm_month']=tm_dt.dt.month
    df['tm_dayofweek']=tm_dt.dt.dayofweek
    df['tm_day']=tm_dt.dt.day
    df['tm_hour']=tm_dt.dt.hour
    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
    df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
    df['tmdlta_twtu2'] = df.tm - df.u2_create_tm

    for typ in twttyps:
        df[f'twttyp_{typ}']=(df.twttyp==typ).astype('int8')

    for lang in langs:
        df[f'lang_{lang}']=(df.lang==lang).astype('int8')
    if istrn: 
        df[cols_tgt]=df[cols_tgt_tmstmp].notna().astype('int8')
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 'twttyp', 'lang', 'twtid', 'u1id', 'u2id']+cols_tgt_tmstmp, )
    else:
        df.drop(inplace=True, columns=['toks', 'hshtgs', 'media', 'links', 'domns', 'twttyp', 'lang', 'u1id',])        
    return df

In [6]:
grand_total/1e6

150.0

In [9]:
chnks_trn = pd.read_csv(f'{p_in}/trn_{trntmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=1e6
                       )
for ichnk,df in enumerate(chnks_trn):
    mintm,maxtm=df.tm.min(),df.tm.max()
    mintm,maxtm=(datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (mintm,maxtm))
    display(df.head(1))
    print(dtnow(), 'chunk', ichnk, mintm,maxtm)
ichnk

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...,NaN,D4D1EBDE74F74C5DA529959AF979625C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,...,0000006C3074607050F1339DDCB890BB,27428,600,False,1520948869,False,NaN,NaN,NaN,NaN


2020-03-20 02:46:19 chunk 0 2020-02-06 00:00:00 2020-02-12 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
1000000,101\t9706\t36553\t108578\t119\t119\t119\t108\t...,11218D05D5E94DE4E577C3AF9338E65D\t3A6FF9E6FA11...,E0B1129F0B31CAE6655127073C08979C,Photo,NaN,NaN,TopLevel,125C57F4FA6D4E110983FB11B52EFD4E,1581348967,30D18739CC163E7629104D16BE4E65DF,...,0DD2A83B6837B1FA23CEA85748175A46,95,354,False,1348229207,False,NaN,1.581355e+09,NaN,NaN


2020-03-20 02:46:26 chunk 1 2020-02-06 00:00:00 2020-02-12 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
2000000,101\t56898\t137\t13672\t11680\t168\t19964\t131...,NaN,947423C12D2EF9A2555210ABF41A3593,Photo,NaN,NaN,Retweet,22C448FF81263D4BAF2A176145EE9EAD,1581468487,276EF70A4F5F7813601EB28CE56E42F6,...,1BAA25B48D84CFB2A6CEE2271D57103C,277,524,False,1361344317,True,NaN,1.581469e+09,NaN,1.581469e+09


...

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
3000000,101\t113\t171\t112\t10176\t22873\t178\t112\t11...,NaN,A355F64BEBCAC5C0C9E1E3E00D088FA9,NaN,NaN,NaN,TopLevel,FA3F382BC409C271E3D6EAF8BE4648DD,1581179275,1703830C32A2559FB06219C75146D1B7,...,297EC923CAD57DACE997A68ED8E47374,84,55,False,1572460222,False,NaN,NaN,NaN,1.581190e+09


2020-03-20 02:46:41 chunk 3 2020-02-06 00:00:00 2020-02-12 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
4000000,101\t8381\t3728\t14813\t35412\t74381\t61963\t1...,NaN,E0CE094934855E6C13DC333D49F88F0C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581376602,D83E1BFDFBAC5816EA663E96484FD688,...,3752CFCF09C4BA0110A7CE0EA6DB4A03,214,281,False,1425402026,True,NaN,NaN,NaN,1.581377e+09


2020-03-20 02:46:48 chunk 4 2020-02-06 00:00:00 2020-02-12 23:59:59


KeyboardInterrupt: 

In [10]:
chnks_val = pd.read_csv(f'{p_in}/val_{valtmstmp}.tsv',sep='\x01',
                    header=None,names=cols, 
                        chunksize=1e6
                       )
for ichnk,df in enumerate(chnks_val):
    mintm,maxtm=df.tm.min(),df.tm.max()
    mintm,maxtm=(datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (mintm,maxtm))
    display(df.head(1))
    print(dtnow(), 'chunk', ichnk, mintm,maxtm)
ichnk

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
0,101\t47185\t10157\t100986\t10343\t55422\t119\t...,NaN,7647B4E9DAF4C1D8973397DC2A04F3E3,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581703126,8A9AB92B775C62C4AB60DF6773A01571,...,0000006C3074607050F1339DDCB890BB,27448,600,False,1520948869,True,NaN,NaN,NaN,NaN


2020-03-20 02:47:18 chunk 0 2020-02-13 00:00:00 2020-02-19 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
1000000,101\t2021\t92499\t10634\t5478\t3350\t4704\t194...,NaN,A6A915BFA9FFA5976CD2BF1A9EED06AE,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581941845,BE001A24ECD05D5960B6980FB85F4A65,...,34EFF535C6BFA34507B20739A70E751D,10,228,False,1549251049,False,NaN,NaN,NaN,NaN


2020-03-20 02:47:29 chunk 1 2020-02-13 00:00:00 2020-02-19 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
2000000,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,BE89706D9107A983AD69B37850C6BEEE,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1582132351,7C7944FB998880F5C04A2398519A9110,...,69F10891EE469B16880EE628F1339A45,132,126,False,1532748723,False,NaN,NaN,NaN,NaN


...

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
10000000,101\t56898\t137\t10189\t34869\t10161\t17025\t1...,NaN,16300981288960E9C8A2809B02966778,Video,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581618284,384A5F15B309BAB4C9FE758D5D3E8FBF,...,A9EF3CCE29C14854D2C3032831E58F99,467,38,False,1361791969,True,NaN,NaN,NaN,NaN


2020-03-20 02:49:24 chunk 10 2020-02-13 00:00:00 2020-02-19 23:59:59


,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,u2id,u2_follower_cnt,u2_following_cnt,u2_vrfed,u2_create_tm,u1_fllw_u2,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
11000000,101\t56898\t137\t10966\t41528\t67646\t14139\t1...,NaN,C4A7BBB7EB872FC04C98D1FB24B6EA82,NaN,NaN,NaN,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581990322,3FADD1A0ABFA2ABFA13E0021652752AC,...,DECDF099ADB7577480CF30890C3D4C24,151,239,False,1481422034,False,NaN,NaN,NaN,NaN


2020-03-20 02:49:35 chunk 11 2020-02-13 00:00:00 2020-02-19 23:59:59


KeyboardInterrupt: 